# Polytop FDB Access

## Installation
See instruction in [Polytope installation](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1)

**Configuring Access to Polytope**

In [ ]:
import os
import dataclasses as dc
from meteodatalab import mars, mch_model_data

#ECMWF polytope credentials
# leave empty for ICON-CSCS polytope
os.environ["POLYTOPE_USER_KEY"] = ""
os.environ["POLYTOPE_USER_EMAIL"] = ""

#ICON-CSCS polytope credentials
# leave empty for ecmwf polytope
os.environ["POLYTOPE_USERNAME"] =
os.environ["POLYTOPE_PASSWORD"] =
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"

**Selecting date,time of the forecast**

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time


**Querying the data**

In [ ]:
request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    number=1,
    step=0
)

In [ ]:
ds = mch_model_data.get_from_polytope(request)

In [ ]:
ds_temp = ds["T_2M"]
lons = ds_temp["lon"].values
lats = ds_temp["lat"].values

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon,Point
import numpy as np

triangle_coords = [
    (8.0, 43.5),  # central point-ish
    (5, 46.7),  # east
    (10.2, 48.9)   # north
]
triangle = Polygon(triangle_coords)


triangle_gdf = gpd.GeoDataFrame(
    {"name": ["triangle"]},
    geometry=[triangle],
    crs="EPSG:4326"
)

points = gpd.GeoSeries.from_xy(lons.ravel(), lats.ravel(), crs="EPSG:4326")

mask = points.within(triangle)
mask_2d = mask.values.reshape(lats.shape)

ds_triangle = ds_temp.where(mask_2d)

ds_triangle.plot()


**Convert to degree celcius**

In [ ]:
temp2m = ds['T_2M']
temp2m_m = temp2m.mean(dim='eps', keep_attrs=True)
temp2m_m.values = temp2m_m.values - 273.15

**Regrid**

In [ ]:
from meteodatalab.operators.regrid import regrid, RegularGrid, Resampling, CRS

dst = RegularGrid(
    crs=CRS.from_epsg(21781),
    xmin=400000,
    xmax=900000,
    ymin=50000,
    ymax=350000,
    nx=500,
    ny=300,
)
t_final = regrid(temp2m_m, dst, Resampling.bilinear)


**Plot the results**

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy import feature as cfeature

crs = ccrs.epsg(21781)
fig = plt.figure(figsize=(11, 8))
ax = fig.add_subplot(1, 1, 1, projection=crs)

f = t_final.squeeze()
c = ax.contourf(dst.x, dst.y, f.values, transform=crs, cmap='RdBu_r')
fig.colorbar(c, label="°C", orientation="horizontal")

ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

ax.set_title("2 Meter Temperature")